## Product Sales Prediction using Time Series

### Using wget/gdown to download CSV file from Google Drive

In [1]:
!gdown 'https://drive.google.com/uc?id=1xcaBvZ_8xWX-X3NgVMaNrjwIJwe7kT7T'

Downloading...
From: https://drive.google.com/uc?id=1xcaBvZ_8xWX-X3NgVMaNrjwIJwe7kT7T
To: /Users/nathalia/Personal/code-practice/Data Science Projects/Sales of Product Prediction - Time Series /sales.csv
100%|██████████████████████████████████████| 1.60k/1.60k [00:00<00:00, 4.93MB/s]


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import read_csv

In [3]:
import os
working_dir = os.getcwd()
print(working_dir)

/Users/nathalia/Personal/code-practice/Data Science Projects/Sales of Product Prediction - Time Series 


In [4]:
data_path = working_dir + '/sales.csv'

### Turn data from CSV file into a DataFrame

In [5]:
df= read_csv(data_path)

# Results
print(f'Dataset shape: {df.shape}')
df.head()

Dataset shape: (107, 2)


,Month,Perrin Freres monthly champagne sales millions ?64-?72
0,1964-01,2815.0
1,1964-02,2672.0
2,1964-03,2755.0
3,1964-04,2721.0
4,1964-05,2946.0


In [6]:
df.columns

Index(['Month', 'Perrin Freres monthly champagne sales millions ?64-?72'], dtype='object')

In [7]:
df.shape

(107, 2)

In [8]:
df.isnull().sum()

Month                                                     1
Perrin Freres monthly champagne sales millions ?64-?72    2
dtype: int64

### Remove file downloaded in the path after finishing with the project

In [9]:
os.remove(data_path)